In [1]:
import pandas as pd
import requests

In [2]:
import random
import numpy as np
import pandas as pd
from scipy.stats import norm

In [3]:
league_id = 384052481958785024
base_url = 'https://api.sleeper.app/v1/league/' + str(league_id) + '/'

## Get Sleeper User-Details

In [4]:
teams = requests.get(base_url + 'users')

In [5]:
team_lst = teams.json()

In [6]:
team_tups = {}

In [7]:
for val in team_lst:
    team_tups[val['display_name']] = val['user_id']

In [8]:
team_tups

{'msubbaiah': '307654863477538816',
 'dhaksubb': '307663062192619520',
 'bhargavesh': '307664367791390720',
 'rbachal': '307942185758621696',
 'GADI': '307946516406890496',
 'SAADMONIUSREX': '309778749241454592',
 'ssudhish10': '309848813416226816',
 'cvalluru': '310592766369361920',
 'mmj762': '332232044425797632',
 'arvindsivak': '463448194244669440'}

In [9]:
df = pd.DataFrame.from_dict(team_tups,orient='index').reset_index()
df.columns = ['user_name','user_id']
df['roster_id'] = 0

In [10]:
df.head()

,user_name,user_id,roster_id
0,msubbaiah,307654863477538816,0
1,dhaksubb,307663062192619520,0
2,bhargavesh,307664367791390720,0
3,rbachal,307942185758621696,0
4,GADI,307946516406890496,0


Add in roster information as well to the lookup dataframe. 

In [11]:
rosters = requests.get(base_url + 'rosters')

In [12]:
rj = rosters.json()

In [13]:
for val in rj:
    df.loc[df['user_id'] == val['owner_id'],'roster_id'] = val['roster_id']

In [14]:
user_roster_map = df.set_index('roster_id').T.to_dict('records')[0]

In [15]:
user_roster_map

{1: 'msubbaiah',
 2: 'dhaksubb',
 3: 'bhargavesh',
 4: 'rbachal',
 5: 'GADI',
 6: 'SAADMONIUSREX',
 8: 'ssudhish10',
 9: 'cvalluru',
 7: 'mmj762',
 10: 'arvindsivak'}

## Get week to week data

In [16]:
def extract_pts(lst,dct=None):
    if(dct is None):
        dct = {}
        for l in lst:
            dct[l['roster_id']] = [l['points']]
    else: 
        for l in lst: 
            dct[l['roster_id']].append(l['points'])
    return(dct)

In [17]:
weeks = range(1,14)

In [25]:
for i in weeks:
    print(i)
    wk = requests.get(base_url + 'matchups/' + str(i))
    wk_vals = wk.json()
    if(i==1):
        res_dct = extract_pts(wk_vals)
        continue
    res_dct = extract_pts(wk_vals,res_dct)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [27]:
def sim(team, team_dict):
    keys = list(team_dict.keys())
    keys.remove(team)
    opponents = np.random.choice(keys,size=13,replace = True)
    wins = 0
    for i, score in enumerate(team_dict[team]):
        if team_dict[opponents[i]][i] < score:
            wins += 1
    return(wins)

In [28]:
def prop_of_reality(team,team_dict):
    results = {}
    for i in range(100000):
        results[i] = sim(team, team_dict)
    SResults = pd.Series(results)
    mu = SResults.mean()
    std = SResults.std()
    print(mu,std)
    p0 = norm.cdf(0, loc=mu, scale = std)
    p1 = norm.cdf(1, loc=mu, scale = std)
    p2 = norm.cdf(2, loc=mu, scale = std)
    p3 = norm.cdf(3, loc=mu, scale = std)
    p4 = norm.cdf(4, loc=mu, scale = std)
    p5 = norm.cdf(5, loc=mu, scale = std)
    p6 = norm.cdf(6, loc=mu, scale=std)
    p7 = norm.cdf(7, loc=mu, scale=std)
    p8 = norm.cdf(8, loc=mu, scale=std)
    p9 = norm.cdf(9, loc=mu, scale=std)
    p10 = norm.cdf(10, loc=mu, scale=std)   
    p11 = norm.cdf(11, loc=mu, scale=std)
    p12 = norm.cdf(12, loc=mu, scale=std)
    p13 = norm.cdf(13, loc=mu, scale=std)
    return(pd.Series({"w0":p0, "w1": p1-p0, "w2": p2-p1, \
                     "w3":p3-p2, "w4":p4-p3,'w5':p5-p4,'w6':p6-p5,'w7':p7-p6,
                     'w8':p8-p7, 'w9':p9-p8, 'w10':p10-p9,'w11':p11-p10,
                     'w12':p12-p11,'w13':1-p13}))

In [29]:
df = pd.DataFrame()

In [30]:
for team in res_dct.keys():
    tm_name = user_roster_map[team]
    print(tm_name)
    df[tm_name] =prop_of_reality(team,res_dct)

msubbaiah
7.22919 1.4235667210644585
dhaksubb
6.66746 1.434711724486101
bhargavesh
5.4468 1.3664071247922605
rbachal
3.78058 1.268215654890719
GADI
6.22342 1.4254416236517746
SAADMONIUSREX
9.33108 1.222939487222664
mmj762
7.4413 1.3813664943578154
ssudhish10
6.4429 1.4589931036881163
cvalluru
7.33133 1.5133516886151561
arvindsivak
5.10991 1.3709225310670967


In [31]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.index =['0-13','1-12','2-11','3-10','4-9','5-8','6-7','7-6','8-5','9-4','10-3','11-2','12-1','13-0']
df

,msubbaiah,dhaksubb,bhargavesh,rbachal,GADI,SAADMONIUSREX,mmj762,ssudhish10,cvalluru,arvindsivak
0-13,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000
1-12,0.000,0.000,0.001,0.013,0.000,0.000,0.000,0.000,0.000,0.001
2-11,0.000,0.001,0.005,0.066,0.001,0.000,0.000,0.001,0.000,0.010
3-10,0.001,0.005,0.031,0.189,0.010,0.000,0.001,0.008,0.002,0.050
4-9,0.010,0.026,0.108,0.300,0.048,0.000,0.006,0.038,0.012,0.147
5-8,0.047,0.091,0.227,0.263,0.136,0.000,0.032,0.114,0.048,0.259
6-7,0.135,0.198,0.285,0.128,0.242,0.003,0.110,0.219,0.128,0.274
7-6,0.242,0.271,0.215,0.034,0.269,0.025,0.226,0.268,0.224,0.174
8-5,0.270,0.232,0.097,0.005,0.187,0.110,0.282,0.208,0.257,0.066
9-4,0.187,0.125,0.026,0.000,0.081,0.255,0.213,0.103,0.194,0.015


In [32]:
df.sum(axis=0)

msubbaiah       1.000
dhaksubb        1.000
bhargavesh      1.000
rbachal         1.000
GADI            1.000
SAADMONIUSREX   0.987
mmj762          1.000
ssudhish10      1.000
cvalluru        0.999
arvindsivak     1.000
dtype: float64

In [33]:
df.to_csv('Sleeper-2019-Luck-Proj.csv')